# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2025-11-09 11:02:02] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2025-11-09 11:02:02] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2025-11-09 11:02:02] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2025-11-09 11:02:04] INFO trace.py:52: opentelemetry package is not installed, tracing disabled


[2025-11-09 11:02:04] WARNING server_args.py:1186: Attention backend not explicitly specified. Use fa3 backend by default.


...

Capturing batches (bs=80 avail_mem=76.28 GB):  25%|██▌       | 5/20 [00:00<00:01,  8.88it/s]

Capturing batches (bs=72 avail_mem=76.27 GB):  35%|███▌      | 7/20 [00:01<00:02,  5.56it/s]

Capturing batches (bs=40 avail_mem=76.25 GB):  50%|█████     | 10/20 [00:01<00:01,  6.78it/s]

Capturing batches (bs=8 avail_mem=76.23 GB):  75%|███████▌  | 15/20 [00:01<00:00, 11.57it/s] 

Capturing batches (bs=1 avail_mem=76.22 GB): 100%|██████████| 20/20 [00:02<00:00,  9.84it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Leo Petytus, and I am a mathematician specializing in abstract algebra and its applications. I have a degree in mathematics from the University of Cambridge, and I have been a resident of the city of Cambridge for the past 20 years. I have a passion for mathematics, and I love solving complex problems and working on intricate puzzles. I am interested in studying mathematical structures and algorithms, and I am eager to learn new things and improve my skills. I have also been interested in the intersection of mathematics and science, and I am fascinated by the way that mathematics can be used to explain the world around us. I enjoy working
Prompt: The president of the United States is
Generated text:  from ________. A. England B. America C. France D. Germany
Answer:
B

Which of the following statements about the protection of minors is incorrect?
A. Promoting traditional virtues
B. Safeguarding minors' rights
C. Improving minors' knowledge
D. A

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name] and I'm a [Age] year old [Gender] [Occupation]. I'm a [Occupation] who has always been passionate about [What interests you about your occupation]. I'm always looking for new challenges and opportunities to grow and learn. I'm a [What do you enjoy doing for fun?] who is always up for a good laugh and enjoys spending time with friends and family. I'm a [What do you like to do for entertainment?] who loves to explore new places and try new foods. I'm a [What do you like to do for exercise?] who is always on the lookout for new

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris, also known as the City of Light, a city with a rich history and diverse culture. It is located on the Seine River and is home to many famous landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum. Paris is also known for its vibrant nightlife, art scene, and delicious cuisine. The city is a major economic and cultural hub in France and is home to many important institutions and organizations. It is a popular tourist destination and is known for its beautiful architecture, charming streets, and lively atmosphere. Paris is a city that is constantly evolving and changing, with new developments and cultural

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by a number of trends that are expected to shape the technology's direction and impact on society. Here are some of the most likely trends:

1. Increased integration with human intelligence: As AI becomes more advanced, it is likely to become more integrated with human intelligence, allowing it to learn and adapt to new situations. This could lead to more sophisticated and adaptive AI systems that can better understand and respond to human emotions and behaviors.

2. Greater use of AI in healthcare: AI is already being used in healthcare to improve patient outcomes and reduce costs. As AI technology continues to improve, it is likely to be used even more



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [Name], and I'm a [Level] employee at [Company]. I'm always looking for ways to improve my skills and stay up-to-date with new technologies. What can you tell me about your background and why you're interested in working at [Company]? You can start by sharing any relevant experiences or achievements that you feel would make you a valuable asset to the company. Additionally, feel free to ask me any questions you may have about [Company]. I'm excited to meet you! Let's chat! [Name] is looking forward to our conversation! [Name] is looking forward to meeting [Name]. What can you tell

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris. 

A. Correct
B. Incorrect
A. Correct

Paris is the capital of France and is known for its rich cultural heritage, iconic landmarks, and historical significance. The city is home

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 __

________

.

 I

...

 This

 could

 lead

 to

 more

In [6]:
llm.shutdown()